In [1]:
import numpy as np
import torch, re
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from __future__ import print_function
import numpy as np
import librosa

def tag_processing_audio(file_type):
    PATH = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/"
    FILE = "translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_46856990.transcription.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_24474003.transcription.txt"

    from pathlib import Path

    pathlist = Path(PATH+file_type).glob('**/*.wav')
    tag_list = []
    #print(pathlist)
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        #print(path_in_str)
        #print(path_in_str.split("/")[-1:][0])
        FILE2 = path_in_str.split("/")[-1:][0]
        
        # 1. Get the file path to the included audio example
        #filename = "/home/va/Kathy/scratch/src/MATERIAL_BASE-1B_10171280.wav"
        
        filename = PATH+file_type + FILE2
        # 2. Load the audio as a waveform `y`
        #    Store the sampling rate as `sr`
        print(filename)
        y, sr = librosa.load(filename, res_type='kaiser_fast')

        # 3. Run the default beat tracker
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0) 

        oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=512)
        tempograms = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=512)
        #tempograms = np.mean(librosa.feature.tempogram(y=y, sr=sr, n_mfcc=40).T,axis=0) 

        #print('Estimated tempo: {:.2f} beats per minute'.format(tempo))
        np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
        #print(mfccs, tempograms)

        print(mfccs.shape, tempograms.shape)

        # 4. Convert the frame indices of beat events into timestamps
        beat_times = librosa.frames_to_time(beat_frames, sr=sr)

        librosa.output.times_csv(PATH+"processed_beats/" + FILE2 + ".txt", beat_times)
        librosa.output.times_csv(PATH+"processed_mfccs/" + FILE2 + ".txt", mfccs) #non-normalized
        #librosa.output.times_csv(PATH+"processed_tempo/" + FILE2 + ".txt", ''.join(x for x in tempograms.tolist())) #tempo
        with open(PATH+"processed_tempo/" + FILE2 + ".txt", 'w') as outfile:
            outfile.write('# Array shape: {0}\n'.format(tempograms.shape))
            # Iterating through a ndimensional array produces slices along
            # the last axis. This is equivalent to data[i,:,:] in this case
            for data_slice in tempograms:
                # The formatting string indicates that I'm writing out
                # the values in left-justified columns 7 characters in width
                # with 2 decimal places.  
                np.savetxt(outfile, data_slice, fmt='%-7.4f')
                # Writing out a break to indicate different slices...
                outfile.write('# New slice\n')
        
        '''
        with open(PATH+file_type + FILE2) as file:
            cnt = 1
            pat = r'(?<=\<).+?(?=\>)'
            #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0])
            for line in file:
                tags = re.findall(pat, line)
        
                for i in range(len(line[:-1])):
                    #print(i)
                    if(line[i] == '<'):
                        new_tag = ""
                        for j in range(i+1, len(line[:-1])):
                            if(line[j] == '>'):
                                i = j
                                if(not new_tag.isdigit() and (new_tag not in tag_list)):
                                    tag_list.append(new_tag)
                                    #print(new_tag)
                                #tag_list[new_tag] = True
                                break
                            else:
                                new_tag = new_tag + line[j]

                cnt = cnt + 1
                #break
            #break
        '''
    #print(tag_list)
    return tag_list

tag_processing_audio("src/")


/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_77097070.wav
(40,) (384, 13960)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_41171885.wav
(40,) (384, 3493)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_88100113.wav
(40,) (384, 5177)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_81447573.wav
(40,) (384, 3362)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_31089786.wav
(40,) (384, 2863)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_57086308.wav
(40,) (384, 3005)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_41581943.wav
(40,) (384, 17564)
/Use

(40,) (384, 2583)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_62980994.wav
(40,) (384, 9807)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_38244746.wav
(40,) (384, 12816)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_23555478.wav
(40,) (384, 8162)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_50599196.wav
(40,) (384, 3644)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_30471970.wav
(40,) (384, 7238)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_24474003.wav
(40,) (384, 10415)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_14493866.wav
(40,

(40,) (384, 9715)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_96279348.wav
(40,) (384, 4908)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_10760160.wav
(40,) (384, 6475)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_52960427.wav
(40,) (384, 10460)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_13843140.wav
(40,) (384, 1659)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_10675424.wav
(40,) (384, 8471)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_79504607.wav
(40,) (384, 7966)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_67137459.wav
(40,)

[]

In [2]:
def tag_processing(file_type):
    PATH = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/"
    FILE = "translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_46856990.transcription.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_24474003.transcription.txt"

    from pathlib import Path

    pathlist = Path(PATH+file_type).glob('**/*.txt')
    tag_list = []
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        #print(path_in_str)
        #print(path_in_str.split("/")[-1:][0])
        FILE2 = path_in_str.split("/")[-1:][0]

        with open(PATH+file_type + FILE2) as file:
            cnt = 1
            pat = r'(?<=\<).+?(?=\>)'
            #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0])
            for line in file:
                tags = re.findall(pat, line)
                '''
                if(len(tags) == 0):
                    print("Len=",  len(tags), line)
                    #print(line[:-1])
                    #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0], tags)
                    continue
                #print("New Tags")
                print(line[:-1])
                '''

                for i in range(len(line[:-1])):
                    #print(i)
                    if(line[i] == '<'):
                        new_tag = ""
                        for j in range(i+1, len(line[:-1])):
                            if(line[j] == '>'):
                                i = j
                                if(not new_tag.isdigit() and (new_tag not in tag_list)):
                                    tag_list.append(new_tag)
                                    #print(new_tag)
                                #tag_list[new_tag] = True
                                break
                            else:
                                new_tag = new_tag + line[j]

                '''
                if("<no-speech>" in line):
                    continue;
                elif("<sta>" in line):
                    print(cnt, "Start: " + line.split("<sta>",1)[1][:-1])
                elif("<int>" in line):
                    print(cnt, "Interrupt: " + line.split("<int>",1)[1][:-1])
                '''
                cnt = cnt + 1
                #break
            #break
    #print(tag_list)
    return tag_list

transcription_tag_list = tag_processing("transcription/")
translation_tag_list = tag_processing("translation/")

transcription_tag_list.sort()
translation_tag_list.sort()

print(transcription_tag_list)
print(translation_tag_list)

['breath', 'cough', 'foreign', 'hes', 'int', 'laugh', 'lipsmack', 'misc', 'no-speech', 'overlap', 'sta']
['breath', 'cough', 'foreign', 'hes', 'int', 'laugh', 'lipsmack', 'misc', 'no-speech', 'overlap', 'sta']


In [ ]:
##############Ignore Kaldi
#Kaldi Swahili dataset

In [ ]:
def collated_data():
    PATH = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/"
    FILE = "translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_46856990.transcription.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_24474003.transcription.txt"

    from pathlib import Path
    
    pathlist = Path(PATH+file_type).glob('**/*.txt')
    tag_list = []
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        #print(path_in_str)
        #print(path_in_str.split("/")[-1:][0])
        FILE2 = path_in_str.split("/")[-1:][0]

        with open(PATH+"translation/" + FILE2) as file1, open(PATH+"transcription/" + FILE2) as file2:
            cnt = 1
            pat = r'(?<=\<).+?(?=\>)'
            #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0])
            for line in file1:
                
                tags = re.findall(pat, line)
                '''
                if(len(tags) == 0):
                    print("Len=",  len(tags), line)
                    #print(line[:-1])
                    #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0], tags)
                    continue
                #print("New Tags")
                print(line[:-1])
                '''

                for i in range(len(line[:-1])):
                    #print(i)
                    if(line[i] == '<'):
                        new_tag = ""
                        for j in range(i+1, len(line[:-1])):
                            if(line[j] == '>'):
                                i = j
                                if(not new_tag.isdigit() and (new_tag not in tag_list)):
                                    tag_list.append(new_tag)
                                    #print(new_tag)
                                #tag_list[new_tag] = True
                                break
                            else:
                                new_tag = new_tag + line[j]

                '''
                if("<no-speech>" in line):
                    continue;
                elif("<sta>" in line):
                    print(cnt, "Start: " + line.split("<sta>",1)[1][:-1])
                elif("<int>" in line):
                    print(cnt, "Interrupt: " + line.split("<int>",1)[1][:-1])
                '''
                cnt = cnt + 1
                #break
            #break
    #print(tag_list)
    return tag_list

#translation_vector_list = collated_data("translation/")

transcription_tag_list.sort()
translation_tag_list.sort()

print(transcription_tag_list)
print(translation_tag_list)

In [ ]:
#import tensorflow as tf
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with open(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

import torchwordemb
str_english = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
#vocabulary = read_data(str_english)
#print 
#print(vocabulary[:7])

str_english = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
#vocabulary = read_data(str_english)
#print(vocabulary[:7])

vocab, vec = torchwordemb.load_word2vec_bin("/Users/va/Courses/Kathy/Word2Vec/word2vec_model.bin")
print(vec.size())
print(vec[w2v.vocab["apple"] ] )


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))  # Gaussian

def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)

class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.sigmoid(self.map2(x))
        return self.map3(x)

class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.elu(self.map2(x))
        return F.sigmoid(self.map3(x))

def extract(v):
    return v.data.storage().tolist()

def stats(d):
    return [np.mean(d), np.std(d)]

def decorate_with_diffs(data, exponent):
    mean = torch.mean(data.data, 1, keepdim=True)
    mean_broadcast = torch.mul(torch.ones(data.size()), mean.tolist()[0][0])
    diffs = torch.pow(data - Variable(mean_broadcast), exponent)
    return torch.cat([data, diffs], 1)

def caller():
    # Data params
    data_mean = 4
    data_stddev = 1.25

    # Model params
    g_input_size = 1     # Random noise dimension coming into generator, per output vector
    g_hidden_size = 50   # Generator complexity
    g_output_size = 1    # size of generated output vector
    d_input_size = 100   # Minibatch size - cardinality of distributions
    d_hidden_size = 50   # Discriminator complexity
    d_output_size = 1    # Single dimension for 'real' vs. 'fake'
    minibatch_size = d_input_size

    d_learning_rate = 2e-4  # 2e-4
    g_learning_rate = 2e-4
    optim_betas = (0.9, 0.999)
    num_epochs = 30000
    print_interval = 200
    d_steps = 1  # 'k' steps in the original GAN paper. Can put the discriminator on higher training freq than generator
    g_steps = 1

    # ### Uncomment only one of these
    #(name, preprocess, d_input_func) = ("Raw data", lambda data: data, lambda x: x)
    (name, preprocess, d_input_func) = ("Data and variances", lambda data: decorate_with_diffs(data, 2.0), lambda x: x * 2)

    print("Using data [%s]" % (name))

    # ##### DATA: Target data and generator input data

    d_sampler = get_distribution_sampler(data_mean, data_stddev)
    gi_sampler = get_generator_input_sampler()
    G = Generator(input_size=g_input_size, hidden_size=g_hidden_size, output_size=g_output_size)
    D = Discriminator(input_size=d_input_func(d_input_size), hidden_size=d_hidden_size, output_size=d_output_size)
    criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
    d_optimizer = optim.Adam(D.parameters(), lr=d_learning_rate, betas=optim_betas)
    g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=optim_betas)

    for epoch in range(num_epochs):
        for d_index in range(d_steps):
            # 1. Train D on real+fake
            D.zero_grad()

            #  1A: Train D on real
            d_real_data = Variable(d_sampler(d_input_size))
            d_real_decision = D(preprocess(d_real_data))
            d_real_error = criterion(d_real_decision, Variable(torch.ones(1)))  # ones = true
            d_real_error.backward() # compute/store gradients, but don't change params

            #  1B: Train D on fake
            d_gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            d_fake_data = G(d_gen_input).detach()  # detach to avoid training G on these labels
            d_fake_decision = D(preprocess(d_fake_data.t()))
            d_fake_error = criterion(d_fake_decision, Variable(torch.zeros(1)))  # zeros = fake
            d_fake_error.backward()
            d_optimizer.step()     # Only optimizes D's parameters; changes based on stored gradients from backward()

        for g_index in range(g_steps):
            # 2. Train G on D's response (but DO NOT train D on these labels)
            G.zero_grad()

            gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            g_fake_data = G(gen_input)
            dg_fake_decision = D(preprocess(g_fake_data.t()))
            g_error = criterion(dg_fake_decision, Variable(torch.ones(1)))  # we want to fool, so pretend it's all genuine

            g_error.backward()
            g_optimizer.step()  # Only optimizes G's parameters

        if epoch % print_interval == 0:
            print("%s: D: %s/%s G: %s (Real: %s, Fake: %s) " % (epoch,
                                                                extract(d_real_error)[0],
                                                                extract(d_fake_error)[0],
                                                                extract(g_error)[0],
                                                            stats(extract(d_real_data)),
                                                            stats(extract(d_fake_data))))
caller()